# data는 e9t(Lucy Park)님께서 github에 공유해주신 네이버 영화평점 데이터를 사용하였습니다.
# https://github.com/e9t/nsmc

In [1]:
from collections import defaultdict

import torch.nn as nn
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import random
import numpy as np

# data를 읽어옴
def read_txt(path_to_file):
    txt_ls = []
    label_ls = []

    with open(path_to_file) as f:
        for i, line in enumerate(f.readlines()[1:]):
            id_num, txt, label = line.split('\t')
            txt_ls.append(txt)
            label_ls.append(int(label.replace('\n','')))
    return txt_ls, label_ls

# 불러오기

In [2]:
from collections import Counter

In [3]:
x_train, y_train = read_txt('../ratings_train.txt')
x_test, y_test = read_txt('../ratings_test.txt')

x_train = [x.split() for x in x_train]
x_test = [x.split() for x in x_test]

In [4]:
x_train[0]

['아', '더빙..', '진짜', '짜증나네요', '목소리']

In [5]:
Counter(y_train)

Counter({0: 75173, 1: 74827})

## 비어있는 리뷰 제거

In [6]:
def remove_empty_review(X, Y):
    empty_idx_ls = []
    
    for idx, review in enumerate(X):
        if len(review) == 0:
            empty_idx_ls.append(idx)
    
    # idx 값이 큰 것부터 제거 (앞으로 밀리는 것을 방지)
    empty_idx_ls = sorted(empty_idx_ls, reverse = True)
    
    for empty_idx in empty_idx_ls:
        del X[empty_idx], Y[empty_idx]
    
    return X, Y

In [7]:
x_train, y_train = remove_empty_review(x_train, y_train)
x_test, y_test = remove_empty_review(x_test, y_test)

In [8]:
x_train[0]

['아', '더빙..', '진짜', '짜증나네요', '목소리']

## Add Padding

In [9]:
# Sequence Length를 맞추기 위한 padding
def add_padding(token_ls, max_len):
    pad = '<PAD>'
    seq_length_ls = []
    
    for i, tokens in enumerate(token_ls):
        seq_length = len(tokens)
        
        # 짧으면 padding을 추가
        if seq_length < max_len:
            seq_length_ls.append(seq_length)
            token_ls[i] += [pad] * (max_len - seq_length)
        
        # 길이가 길면, max_len까지의 token만 사용
        elif seq_length >= max_len:
            seq_length_ls.append(max_len)
            token_ls[i] = tokens[:max_len]
            
    return token_ls, seq_length_ls


In [10]:
max_sequence_length = 30
x_train, x_train_seq_length = add_padding(x_train, max_sequence_length)
x_test, x_test_seq_length = add_padding(x_test, max_sequence_length)

## Converting token to index 

In [11]:
# 단어에 대한 idx 부여
def convert_token_to_idx(token_ls):
    for tokens in token_ls:
        yield [token2idx[token] for token in tokens]
    return

In [12]:
token2idx = defaultdict(lambda: len(token2idx))
pad = token2idx['<PAD>']

x_train = list(convert_token_to_idx(x_train))
x_test = list(convert_token_to_idx(x_test))

idx2token = {val : key for key,val in token2idx.items()}

In [13]:
' '.join([idx2token[x] for x in x_train[0]])

'아 더빙.. 진짜 짜증나네요 목소리 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>'

### Packing을 위해, 긴 문장이 위로 오도록 순서를 sorting and Variable로 변환

In [16]:
def sort_by_sequence_length(x, y, seq_len):
    sorted_idx = np.argsort(np.array(seq_len))[::-1]
    
    x = Variable(torch.LongTensor(np.array(x)[sorted_idx]))
    y = Variable(torch.LongTensor(np.array(y)[sorted_idx]))
    seq_len = Variable(torch.LongTensor(np.array(seq_len)[sorted_idx]))
    
    return x, y, seq_len

In [32]:
x_train, y_train, x_train_seq_length = sort_by_sequence_length(x_train, y_train, x_train_seq_length)
x_test, y_test, x_test_seq_length = sort_by_sequence_length(x_test, y_test, x_test_seq_length)

# LSTM

In [21]:
class LSTM(nn.Module):
    def __init__(self, token2idx, max_sequence, vocab_size, embed_size, hid_size, n_layers, dropout, bidirectional, n_category):
        super(LSTM, self).__init__() 
        self.vocab_size = vocab_size                # 고유한 단어의 수
        self.embed_size = embed_size                # 임베딩 차원의 크기
        self.padding_index = token2idx['<PAD>']     # 패딩 토큰
        
        self.embed = nn.Embedding(
            num_embeddings=vocab_size, 
            embedding_dim=embed_size, 
            padding_idx=self.padding_index
        )
        
        self.max_sequence = max_sequence           # 한 문장의 최대 길이
        self.hid_size = hid_size                   # LSTM 뉴런의 갯수
        self.n_layers = n_layers                   # LSTM layer의 수
        self.drouput = dropout                     # 드롭아웃
        self.n_category = n_category               # 분류 카테고리 갯수
        self.bidirectional = bidirectional         # optinal, True : bidirectional 
        
        self.lstm = nn.LSTM(embed_size, hid_size, n_layers, batch_first=True, bidirectional=True)
        
        if bidirectional:
            input_dim = 2*hid_size*max_sequence
        else:
            input_dim = hid_size*max_sequence

        self.lin = nn.Linear(input_dim, n_category)
        self.outputs = []
        
    def init_hidden(self, batch_size):
        # 최초에 h_0와 c_0의 초기값 부여 (n_layers, batch_size, hid_size(n_neuron))
        # bidirectional일 경우, (2*n_layers, batch_size, hid_size)
        if self.bidirectional:
            h_0 = Variable(torch.randn(2*self.n_layers, batch_size, self.hid_size)) * 0.1
            c_0 = Variable(torch.randn(2*self.n_layers, batch_size, self.hid_size)) * 0.1
        else:
            h_0 = Variable(torch.randn(self.n_layers, batch_size, self.hid_size)) * 0.1
            c_0 = Variable(torch.randn(self.n_layers, batch_size, self.hid_size)) * 0.1
        return (h_0, c_0)    
    
    def forward(self, x, x_sequence_length):
        # init h randomly
        batch_size = x.size(0)
        self.h_c = self.init_hidden(batch_size)
        
        # embedding
        x = self.embed(x) # sequence_length(max_len), batch_size, embed_size
        
        # packing for LSTM
        x = torch.nn.utils.rnn.pack_padded_sequence(x, x_sequence_length, batch_first=True)
        
        # LSTM
        output, self.h_c = self.lstm(x, self.h_c)
        
        # unpack
        # unpacking 과정에서, size(1) S가 가장 긴 sequence 길이를 갖는 data에 맞춰진다.
        # S가 max_sequence_length로 고정되는 것이 아님을 주의
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(output, batch_first=True)
        #print(x.shape)
        
        # flatten
        x = x.contiguous()
        x = x.view(batch_size, -1)
        
        # fully-connect
        logit = self.lin(x)
        
        return logit
        

In [26]:
params = {
    'token2idx' : token2idx,
    'max_sequence' : 30,
    'vocab_size' : len(token2idx),
    'embed_size' : 32,
    'hid_size' : 32,
    'n_layers' : 2,
    'dropout' : 0.5,
    'bidirectional' : True,
    'n_category' : 2,
}

In [28]:
model = LSTM(**params)

In [29]:
model

LSTM(
  (embed): Embedding(448188, 32, padding_idx=0)
  (lstm): LSTM(32, 32, num_layers=8, batch_first=True, bidirectional=True)
  (lin): Sequential(
    (0): Linear(in_features=1920, out_features=2, bias=True)
    (1): Tanh()
    (2): Dropout(p=0.5)
  )
)

# Train

In [30]:
def adjust_learning_rate(optimizer, epoch, init_lr=0.001, lr_decay_epoch=10):
    """Decay learning rate by a factor of 0.1 every lr_decay_epoch epochs."""
    lr = init_lr * (0.1**(epoch // lr_decay_epoch))

    if epoch % lr_decay_epoch == 0:
        print('LR is set to %s'%(lr))

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return optimizer

In [ ]:
epochs = 50
lr = 0.01
batch_size = 10000

train_idx = np.arange(x_train.size(0))
test_idx = np.arange(x_test.size(0))
optimizer = torch.optim.Adam(model.parameters(),lr)
criterion = nn.CrossEntropyLoss(reduction='sum')

loss_ls = []

for epoch in range(1, epochs+1):
    model.train()
    
    # input 데이터 순서 섞기
    random.shuffle(train_idx)
    x_train, y_train = x_train[train_idx], y_train[train_idx]
    x_train_seq_length = x_train_seq_length[train_idx]
    
    train_loss = 0

    for start_idx, end_idx in zip(range(0, x_train.size(0), batch_size),
                                  range(batch_size, x_train.size(0)+1, batch_size)):
        # batch 뽑기
        x_batch = x_train[start_idx : end_idx]
        y_batch = y_train[start_idx : end_idx].long()
        x_batch_seq_length = x_train_seq_length[start_idx: end_idx]
        
        # sequence 순서대로 정렬하기
        x_batch, y_batch, x_batch_seq_length = sort_by_sequence_length(x_batch, y_batch, x_batch_seq_length)
        
        scores = model(x_batch, x_batch_seq_length)
        predict = F.softmax(scores, dim=1).argmax(dim = 1)
        
        acc = (predict == y_batch).sum().item() / batch_size
        
        loss = criterion(scores, y_batch)
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print('Train epoch : %s,  loss : %s,  accuracy :%.3f'%(epoch, train_loss / batch_size, acc))
    print('=================================================================================================')
    
    loss_ls.append(train_loss)
    optimizer = adjust_learning_rate(optimizer, epoch, lr, 10) # adjust learning_rate while training
    
    if (epoch) % 5 == 0:
        model.eval()
        scores = model(x_test, x_test_seq_length)
        predict = F.softmax(scores, dim=1).argmax(dim = 1)
        
        acc = (predict == y_test.long()).sum().item() / y_test.size(0)
        loss = criterion(scores, y_test.long())
        
        print('*************************************************************************************************')
        print('*************************************************************************************************')
        print('Test Epoch : %s, Test Loss : %.03f , Test Accuracy : %.03f'%(epoch, loss.item()/y_test.size(0), acc))
        print('*************************************************************************************************')
        print('*************************************************************************************************')
